In [ ]:
import pandas as pd 
import yfinance as yf
from pykrx import stock
from datetime import timedelta
import time
import warnings
# 경고를 무시하도록 설정
warnings.filterwarnings("ignore")
#stock.get_market_cap('20210501','20210510',"000480")
df= pd.read_csv('../4일/data/자산,자본,부채,매출.csv',encoding='cp949')

In [ ]:
#인덱스가 티커인거 
stock.get_market_cap("2019-04-01")
# 걍 이거 아래나 위나 다 일정시간 이상 접근하면 코드가 막히는 문제가 발생함
# https://github.com/sharebook-kr/pykrx/issues 확인할경우 걍 진짜 한국거래소에서 제한 걸어버린거임

In [ ]:
#인덱스 날짜인거 
#주의할점은 31일로 끝날짜 지정시 걍 값이 안불러와질 수 있음 
stock.get_market_cap("2019-04-01", "2019-04-30", "058820")

In [ ]:
stock.get_market_cap_by_date('20210501','20210510', "005930")
# 특정 거래소코드 시가총액 가져오기. 이게 안된다고 하셨음 

In [ ]:
import re

strs = ['(*)', '(IFRS연결)', '(천원)', '(IFRS)', '[제조]',
         '*','(-)',('결손금'),'(수익)','(보고서기재)','()','(정상)']

for s in strs:
    escape_s = re.escape(s)  # 문자열 이스케이프 처리 << 이거 안해줄경우 오류날 수 있음 import re 필수
    #이거 해줘야만 하는 이유는 ()가 특정 그룹처리가 되는데, 문자열로 읽으라고 명령하고 있는거임
    #이거 vscode는 안해줘도 되는데 주피터는 또 해줘야됨 아 ㅋㅋ
    print(escape_s)
    print(s)

### 복습용


- data폴더의 '자산,자본,부채,매출.csv'파일 df로 읽기
- 연결이 붙은 열을 추출해 개별 df로 만들기 (filtered_df) 그후 filtered_df에 동일한 이름의 df의 회사명, 거래소코드, 회계년도를 할당하기
- 기존의 df에서 열이름에 '연결'이 붙은 열을 드랍하기
- 두 데이터프레임의 열 이름을 깔끔하게 정리하기 (for문 사용, 제거할 내용을 리스트화하기)
- 기존 df와 filtered_df를 합치기. 이 때, filtered_df가 연결값이므로 우선시 되어야함 이름은 df_sum으로

In [ ]:
import pandas as pd

df = pd.read_csv("./data/자산,자본,부채,매출.csv",encoding="cp949")
df.head(5)

- 연결이 붙은 열을 추출해 개별 df로 만들기 (filtered_df) 그후 filtered_df에 동일한 이름의 df의 회사명, 거래소코드, 회계년도를 할당하기

In [ ]:
filterd_df = df.filter(regex='연결')
filterd_df

In [ ]:
filterd_df[['회사명','거래소코드','회계년도']]=df[['회사명','거래소코드','회계년도']]
filterd_df.head(5)

In [ ]:
df = df.drop(df.filter(regex='연결').columns, axis=1)
df

In [ ]:
strs = ['(*)(IFRS연결)(천원)', '(수익)(*)(IFRS연결)(천원)' ,'(*)(IFRS)(천원)', '(수익)(*)(IFRS)(천원)','(수익)']

for s in strs:

    filterd_df.columns = filterd_df.columns.str.replace(s, "")
    df.columns = df.columns.str.replace(s, "")
    
filterd_df.columns =filterd_df.columns.str.strip()
filterd_df.columns

In [ ]:
df.columns =df.columns.str.strip()
df.columns

In [ ]:
df_sum = filterd_df.combine_first(df)
df_sum

### 문제1

- df_sum의 회계년도 열을 데이트타임으로 변경하기 
- 매수년도라는 열을 새롭게 만들고, 회계년도 에서 1년뒤 값을 넣기 (년도값만들어감)
- 년도값만 들어가 있는 매수년도 열을 str타입으로 바꾸고 '-04-01' 붙이기. 
- timedelta(import 필요)를 이용해 매수년도 값에 5일을 더해준 매수년도2열 만들기
- df_sum의 거래소코드를 제로패딩해주기 

In [ ]:

df_sum['회계년도'] = pd.to_datetime(df_sum['회계년도'])
df_sum

- 매수년도라는 열을 새롭게 만들고, 회계년도 에서 1년뒤 값을 넣기 (년도값만들어감)
- 년도값만 들어가 있는 매수년도 열을 str타입으로 바꾸고 '-04-01' 붙이기. 
- timedelta(import 필요)를 이용해 매수년도 값에 5일을 더해준 매수년도2열 만들기
- df_sum의 거래소코드를 제로패딩해주기 

In [ ]:
df_sum['매수년도'] = df_sum['회계년도'].dt.year + 1
df_sum['매수년도'] = pd.to_datetime(df_sum['매수년도'].astype(str)+ '-04-01')
df_sum

In [ ]:
df_sum['매수년도2'] = df_sum['매수년도']+timedelta(days=4)
df_sum

In [ ]:
df_sum['거래소코드'] =df_sum['거래소코드'].astype(str).str.zfill(6)
df_sum

### 문제2 df = df_sum.head()로 사용

- 거래소코드로 시가총액 불러오기
- 설명코드를 보고 for in zip을 사용해 시가총액 가져오기
- 시가총액 리스트를 만들고, 해당 리스트를 시가총액열에 넣어주기
- 현재 pykrx를 krx에서 차단해서 사용불가능 걍 코드 보여드리고 설명만해라 

In [ ]:
### 설명용
df=df_sum.head()

#stock.get_market_cap(df['매수년도'][0], df['매수년도2'][0], df['거래소코드'][0])

In [ ]:
stock.get_market_cap_by_date('20240301','20240310', "005930")

In [ ]:
### 설명용2
for i,j,c in zip(df['매수년도'],df['매수년도2'] ,df['거래소코드']):
    print(i, j, c)

In [ ]:
marcap = []

for i,j,c in zip(df['매수년도'],df['매수년도2'] ,df['거래소코드']):
    marcap.append(stock.get_market_cap_by_date(i,j,c).head(1)['시가총액'][0])

df['시가총액'] = marcap
df

In [ ]:
# 설명용

marcap=[]


for i,j,c in zip(df['매수년도'],df['매수년도2'] ,df['거래소코드']):
    marcap.append(stock.get_market_cap(i, j, c).head(1)['시가총액'][0])

df['시가총액'] =marcap
    

### pykrx는 현재 서버에서 차단중이므로 대안구하기

- 1. https://github.com/FinanceData/marcap <<깃허브 사용을 통한 시가총액 가져오기>> 파일이 너무큼
- 2. http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101
- krx에서 일일이 가져온 뒤, merge나 concat써서 사용하기. 우선은 이거로 진행


### concat연습하기
- concat은 위아래로 붙이기

In [ ]:
# 예시(2번으로 일일이 데이터 가져올 때)
df=pd.read_csv('./data/시가총액/2019.csv',encoding='cp949')
df['year'] = 2019 # 따로 년도열을 만들고 열 전체에 2019값을 다 넣겠다
df2=pd.read_csv('./data/시가총액/2020.csv',encoding='cp949')
df2['year'] = 2020
df3=pd.concat([df, df2])
df3.sort_values('종목코드')

### 문제3 시가총액 무식하게 읽기
목적은 data폴더안의 시가총액폴더 안 파일을 모두 합치고, 연도를 구분해주는 것 
- for문을 이용
- 저장할 빈 df_marcap이라는 df만들기
- file에 파일명 할당하기
- df로 csv읽기
- df['year']에 해당년도값 할당하기
- df와 df_marcap을 concat하기 
- 거래소코드열을 만들고 종목코드 값 넣기
- 필요한 거래소코드, 시가총액, year 열만 남기고 df_marcap에 저장하기


In [ ]:
years = [2018,2019,2020,2021,2022,2023,2024]
df_marcap = pd.DataFrame() # 빈 데이터 프레임 만들기

for i in years:
    file = f'../4일/data/시가총액/{i}.csv'
    #print(file)
    df = pd.read_csv(file, encoding='cp949')
    df['year'] = i
    df_marcap = pd.concat([df,df_marcap])

df_marcap['거래소코드'] = df_marcap['종목코드']    
df_marcap = df_marcap[['거래소코드', '시가총액', 'year']]
df_marcap


# 문제 4
- df_marcap의 거래소코드, year을 기준으로 df_marcap정렬하기
- df_sum에서 회계년도열의 월 값이 12가 아닌 경우를 출력하기
- 월 값이 12가 아닌 거래소코드 데이터를 모두 버리고 df_12에 저장하기 (isin사용)
- df_12에 'year'열만들기. 매수년도열의 년도값과 동일한 값 넣기 
- 이후 merge를 통해 df_12와 df_marcap합치기. 이때 기준은 거래소코드, year 두 열로 하기 (final_df로 할당)


In [ ]:
df_marcap = df_marcap.sort_values(['거래소코드','year'])
df_marcap

In [ ]:
df_sum

In [ ]:
df_sum[df_sum['회계년도'].dt.month!=12]

In [ ]:
counts = df_sum['거래소코드'].value_counts()

df_12 = df_sum[df_sum['거래소코드'].isin(counts[counts == 6].index)]

In [ ]:
df_12['year'] = df_sum['매수년도'].dt.year

In [ ]:
df_12

In [67]:
final_df = pd.merge(df_12,df_marcap, left_on=['거래소코드','year'], right_on=['거래소코드','year'], how='left')
final_df

,거래소코드,매출액,부채,자본,자산,회계년도,회사명,매수년도,매수년도2,year,시가총액
0,058820,49864782.0,10398033.0,103981846.0,114379880.0,2018-12-01,(주)CMG제약,2019-04-01,2019-04-05,2019,4.981591e+11
1,058820,58622681.0,12084883.0,180158283.0,192243166.0,2019-12-01,(주)CMG제약,2020-04-01,2020-04-05,2020,4.993176e+11
2,058820,64459891.0,14224578.0,181131004.0,195355583.0,2020-12-01,(주)CMG제약,2021-04-01,2021-04-05,2021,6.111259e+11
3,058820,69475496.0,14472099.0,184702494.0,199174593.0,2021-12-01,(주)CMG제약,2022-04-01,2022-04-05,2022,4.764004e+11
4,058820,82197410.0,28307361.0,184953015.0,213260376.0,2022-12-01,(주)CMG제약,2023-04-01,2023-04-05,2023,2.854236e+11
...,...,...,...,...,...,...,...,...,...,...,...
10603,003280,102166838.0,400043339.0,30919383.0,430962722.0,2019-12-01,흥아해운(주),2020-04-01,2020-04-05,2020,2.992840e+10
10604,003280,88938838.0,313059823.0,-26283865.0,286775958.0,2020-12-01,흥아해운(주),2021-04-01,2021-04-05,2021,3.011700e+10
10605,003280,81724314.0,144358895.0,102939788.0,247298683.0,2021-12-01,흥아해운(주),2022-04-01,2022-04-05,2022,6.852110e+11
10606,003280,177860078.0,181755200.0,135304736.0,317059937.0,2022-12-01,흥아해운(주),2023-04-01,2023-04-05,2023,3.315459e+11


In [68]:
final_df['거래소코드'].value_counts()

거래소코드
058820    6
194480    6
041930    6
282690    6
170900    6
         ..
045060    6
122640    6
015360    6
012160    6
003280    6
Name: count, Length: 1768, dtype: int64